In [2]:
#Import Libraries
from IPython.core.display import display
from pyspark.shell import spark
from cassandra.cluster import Cluster
from pyspark import SparkConf
from pyspark.sql import SparkSession
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
# Spark configuration
sparkConf=SparkConf().setMaster("local[3]")\
                     .setAppName("project")\
                     .setAll([('spark.executor.memory','2g'),
                            ('spark.sql.debug.maxToStringFields','1000'),
                            ('spark.driver.memory','2g')])

spark=SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [3]:
# Connection with Cassandra db
cluster = Cluster(['127.0.0.1'], port= 9042)
session = cluster.connect('mykeyspace')

In [4]:
# Reading all the 10 years CSV
year2005 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2005_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2006 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2006_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2007 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2007_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2008 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2008_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2009 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2009_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2010 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2010_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2011 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2011_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2012 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2012_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2013 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2013_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2014 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2014_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2015 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2015_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())

In [5]:
# Merging all 11 years data into dataframe
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

MergeData = unionAll(year2005,
                     year2006,
                     year2007,
                     year2008,
                     year2009,
                     year2010,
                     year2011,
                     year2012,
                     year2013,
                     year2014,
                     year2015)

In [6]:
# Count of MergeData
MergeData.count()

27720673

In [7]:
# Show MergeData schema
MergeData.printSchema()

root
 |-- resident_status: integer (nullable = true)
 |-- education_1989_revision: integer (nullable = true)
 |-- education_2003_revision: integer (nullable = true)
 |-- education_reporting_flag: integer (nullable = true)
 |-- month_of_death: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age_type: integer (nullable = true)
 |-- detail_age: integer (nullable = true)
 |-- age_substitution_flag: integer (nullable = true)
 |-- age_recode_52: integer (nullable = true)
 |-- age_recode_27: integer (nullable = true)
 |-- age_recode_12: integer (nullable = true)
 |-- infant_age_recode_22: integer (nullable = true)
 |-- place_of_death_and_decedents_status: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: integer (nullable = true)
 |-- current_data_year: integer (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: integer (nullable = true)
 |-- method_of_disposition: string (nullable = tr

In [5]:
# Create SQL Table from MergeData
MergeData.registerTempTable("mergedTable")

In [9]:
# Query 1 - Male vs female deaths by resident status
results_male_female_resident_status = spark.sql(
  """SELECT resident_status,
            sex,
            count(sex) AS sex_counts
     FROM mergedTable
     GROUP BY resident_status, sex
     ORDER BY resident_status, sex""")

results_male_female_resident_status.show()

+---------------+---+----------+
|resident_status|sex|sex_counts|
+---------------+---+----------+
|              1|  F|  11419032|
|              1|  M|  11030268|
|              2|  F|   2027162|
|              2|  M|   2288859|
|              3|  F|    398187|
|              3|  M|    506845|
|              4|  F|     16495|
|              4|  M|     33825|
+---------------+---+----------+



In [10]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_male_female_resident_status (ind int primary key, resident_status int, sex varchar, sex_counts int)")
stmt = session.prepare("INSERT INTO results_male_female_resident_status (ind, resident_status, sex, sex_counts) VALUES (?, ?, ?, ?)")

for ind, item in results_male_female_resident_status.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['resident_status'], item['sex'], item['sex_counts']])

In [11]:
# Query 2 - Male vs female deaths by month of the year
results_male_deaths_month = spark.sql(
  """SELECT month_of_death,
            sex,
            count(sex) AS sex_counts
     FROM mergedTable
     GROUP BY month_of_death, sex
     ORDER BY month_of_death, sex""")

results_male_deaths_month.show()

+--------------+---+----------+
|month_of_death|sex|sex_counts|
+--------------+---+----------+
|             1|  F|   1302416|
|             1|  M|   1269199|
|             2|  F|   1177388|
|             2|  M|   1147291|
|             3|  F|   1262972|
|             3|  M|   1231322|
|             4|  F|   1150376|
|             4|  M|   1145743|
|             5|  F|   1134538|
|             5|  M|   1144327|
|             6|  F|   1066368|
|             6|  M|   1087923|
|             7|  F|   1088399|
|             7|  M|   1116795|
|             8|  F|   1083513|
|             8|  M|   1107852|
|             9|  F|   1065444|
|             9|  M|   1081367|
|            10|  F|   1144472|
|            10|  M|   1147902|
+--------------+---+----------+
only showing top 20 rows



In [12]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_male_deaths_month (ind int primary key, month_of_death int, sex varchar, sex_counts int)")
stmt = session.prepare("INSERT INTO results_male_deaths_month (ind, month_of_death, sex, sex_counts) VALUES (?, ?, ?, ?)")

for ind, item in results_male_deaths_month.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['month_of_death'], item['sex'], item['sex_counts']])

In [6]:
# Read Disease description CSV (ICD10)
icd10 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/codes/ICD10.csv',
                        header=True,
                        inferSchema=True,
                        ignoreLeadingWhiteSpace=True,
                        ignoreTrailingWhiteSpace=True).cache())

In [7]:
# Create SQL Table of disease description
icd10.registerTempTable("icd10")

In [15]:
# Query 3 - Top 20 diseases causing deaths for either sex
results_diseases_causing_deaths_sex = spark.sql(
  """SELECT m.sex AS sex,
            m.icd_code_10th_revision AS code,
            i.description3 AS description,
            count(m.sex) AS sex_counts
     FROM mergedTable m, icd10 i
     WHERE i.code3 = m.icd_code_10th_revision
     GROUP BY i.description3 ,m.icd_code_10th_revision, m.sex
     ORDER BY count(m.sex) DESC, m.sex
     LIMIT 20""")

results_diseases_causing_deaths_sex.show()

+---+----+--------------------+----------+
|sex|code|         description|sex_counts|
+---+----+--------------------+----------+
|  M|I219|Acute myocardial ...|    765648|
|  F|G309| Alzheimer's disease|    626180|
|  F|I219|Acute myocardial ...|    622679|
|  F|J449|Other chronic obs...|    602655|
|  M|J449|Other chronic obs...|    534241|
|  M| C61|Malignant neoplas...|    311877|
|  F|J189|Pneumonia, unspec...|    276400|
|  M|G309| Alzheimer's disease|    270020|
|  M|J189|Pneumonia, unspec...|    236009|
|  M|C189|Malignant neoplas...|    228669|
|  F|C189|Malignant neoplas...|    223213|
|  M|C259|Malignant neoplas...|    203498|
|  F|C259|Malignant neoplas...|    199666|
|  F|A419|Sepsis, unspecifi...|    199493|
|  M|A419|Sepsis, unspecifi...|    169382|
|  M| G20| Parkinson's disease|    145311|
|  M|I119|Hypertensive hear...|    138554|
|  M|C159|Malignant neoplas...|    123748|
|  F| I10|Essential (primar...|    116720|
|  M|C679|Malignant neoplas...|    114029|
+---+----+-

In [16]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_diseases_causing_deaths_sex (ind int primary key, code varchar, description varchar, sex varchar, sex_counts int)")
stmt = session.prepare("INSERT INTO results_diseases_causing_deaths_sex (ind, code, description, sex, sex_counts) VALUES (?, ?, ?, ?, ?)")

for ind, item in results_diseases_causing_deaths_sex.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['code'], item['description'], item['sex'], item['sex_counts']])

In [18]:
# Query 4 - Method of Disposition
results_disposition = spark.sql(
  """SELECT current_data_year AS year,
            CASE method_of_disposition
            WHEN 'C' THEN 'Cremation'
            WHEN 'B' THEN 'Burial'
            WHEN 'D'THEN 'Donation'
            WHEN 'E' THEN 'Entombment'
            WHEN 'O' THEN 'Other'
            WHEN 'R' THEN 'RemovedFromUSA'
            WHEN 'U' THEN 'Unknown'
            END AS disposition,
            COUNT(*) AS counts
     FROM mergedTable
     GROUP BY 1, 2
     ORDER BY 1, 3""")

results_disposition.show()

+----+--------------+-------+
|year|   disposition| counts|
+----+--------------+-------+
|2005|         Other|   2199|
|2005|      Donation|   4795|
|2005|    Entombment|  21247|
|2005|RemovedFromUSA|  31954|
|2005|     Cremation| 350018|
|2005|        Burial| 553202|
|2005|       Unknown|1489091|
|2006|         Other|   2252|
|2006|      Donation|   6883|
|2006|    Entombment|  23412|
|2006|RemovedFromUSA|  40870|
|2006|     Cremation| 423282|
|2006|        Burial| 667169|
|2006|       Unknown|1266857|
|2007|         Other|   3119|
|2007|      Donation|   8719|
|2007|    Entombment|  26139|
|2007|RemovedFromUSA|  41411|
|2007|     Cremation| 472220|
|2007|        Burial| 725666|
+----+--------------+-------+
only showing top 20 rows



In [19]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_disposition (ind int primary key, counts int, disposition varchar, year int)")
stmt = session.prepare("INSERT INTO results_disposition (ind, counts, disposition, year) VALUES (?, ?, ?, ?)")

for ind, item in results_disposition.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['counts'], item['disposition'], item['year']])

In [21]:
# Query 5 - Manner of death per month
results_deaths_month = spark.sql(
  """SELECT month_of_death AS month,
            CASE manner_of_death
            WHEN '0' THEN 'Not Specified'
            WHEN '1' THEN 'Accident'
            WHEN '2' THEN 'Suicide'
            WHEN '3' THEN 'Homicide'
            WHEN '4' THEN 'Pending investigation'
            WHEN '5' THEN 'Could not be determine'
            WHEN '6' THEN 'Self-Inflicted'
            WHEN '7' THEN 'Natural'
            ELSE 'OTHER'
            END AS manner_death,
            COUNT(*) AS counts
     FROM mergedTable
     GROUP BY 1, 2
     ORDER BY 1,2""")

results_deaths_month.show()

+-----+--------------------+-------+
|month|        manner_death| counts|
+-----+--------------------+-------+
|    1|            Accident| 110563|
|    1|Could not be dete...|  10808|
|    1|            Homicide|  16172|
|    1|             Natural|1859004|
|    1|               OTHER| 536266|
|    1|Pending investiga...|   4261|
|    1|             Suicide|  34541|
|    2|            Accident| 101932|
|    2|Could not be dete...|   9667|
|    2|            Homicide|  13474|
|    2|             Natural|1677102|
|    2|               OTHER| 487921|
|    2|Pending investiga...|   3517|
|    2|             Suicide|  31066|
|    3|            Accident| 110882|
|    3|Could not be dete...|  10421|
|    3|            Homicide|  15866|
|    3|             Natural|1799655|
|    3|               OTHER| 517966|
|    3|Pending investiga...|   3660|
+-----+--------------------+-------+
only showing top 20 rows



In [23]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_deaths_month (ind int primary key, counts int, manner_death varchar, month int)")
stmt = session.prepare("INSERT INTO results_deaths_month (ind, counts, manner_death, month) VALUES (?, ?, ?, ?)")

for ind, item in results_deaths_month.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['counts'], item['manner_death'], item['month']])

In [24]:
# Query 6 - Analysis about Alzheimer's death order by group age
results_alzheimer = spark.sql(
  """SELECT count(*) as deaths_counts,
            CASE age_recode_12
            WHEN '10' THEN '75 - 84 years'
            WHEN '11' THEN '85 years and over'
            WHEN '12' THEN 'Age not stated'
            WHEN '01' THEN 'Under 1 year'
            WHEN '02' THEN '1 - 4 years'
            WHEN '03' THEN '5 - 14 years'
            WHEN '04' THEN '15 - 24 years'
            WHEN '05' THEN '25 - 34 years'
            WHEN '06' THEN '35 - 44 years'
            WHEN '07' THEN '45 - 54 years'
            WHEN '08' THEN '55 - 64 years'
            WHEN '09' THEN '65 - 74 years'
            END AS age
     FROM mergedTable
     WHERE 113_cause_recode = '052'
     GROUP BY age
     ORDER BY deaths_counts DESC
     LIMIT 10""")

results_alzheimer.show()

+-------------+-----------------+
|deaths_counts|              age|
+-------------+-----------------+
|       595395|85 years and over|
|       267194|    75 - 84 years|
|        48876|    65 - 74 years|
|         8598|    55 - 64 years|
|         1082|    45 - 54 years|
|          100|    35 - 44 years|
|           11|   Age not stated|
|            8|    25 - 34 years|
|            1|    15 - 24 years|
+-------------+-----------------+



In [25]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_alzheimer (ind int primary key, age varchar, deaths_counts int)")
stmt = session.prepare("INSERT INTO results_alzheimer (ind, age, deaths_counts) VALUES (?, ?, ?)")

for ind, item in results_alzheimer.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['age'], item['deaths_counts']])

In [26]:
# Query 7 - Count number of graduate people who deads with suicide manner
results_degree_suicides = spark.sql(
  """SELECT count(*) as deaths_counts,
            CASE education_2003_revision
            WHEN '1' THEN '8th grade or less'
            WHEN '2' THEN '9 - 12th grade, no diploma'
            WHEN '3' THEN 'high school graduate or GED completed'
            WHEN '4' THEN 'some college credit, but no degree'
            WHEN '5' THEN 'Associate degree'
            WHEN '6' THEN 'Bachelor’s degree'
            WHEN '7' THEN 'Master’s degree'
            WHEN '8' THEN 'Doctorate or professional degree'
            WHEN '9' THEN 'Unknown'
            END AS education
     FROM mergedTable
     WHERE manner_of_death = '2'
     GROUP BY education
     ORDER BY deaths_counts DESC
     LIMIT 10""")

results_degree_suicides.show()

+-------------+--------------------+
|deaths_counts|           education|
+-------------+--------------------+
|       120629|                null|
|       117887|high school gradu...|
|        51558|some college cred...|
|        37228|9 - 12th grade, n...|
|        35521|   Bachelor’s degree|
|        20697|    Associate degree|
|        12646|   8th grade or less|
|        11644|     Master’s degree|
|         9628|             Unknown|
|         5923|Doctorate or prof...|
+-------------+--------------------+



In [27]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_degree_suicides (ind int primary key, deaths_counts int, education varchar)")
stmt = session.prepare("INSERT INTO results_degree_suicides (ind, deaths_counts, education) VALUES (?, ?, ?)")

for ind, item in results_degree_suicides.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['deaths_counts'], item['education']])

In [28]:
#Columns of MergeData
MergeData.columns

['resident_status',
 'education_1989_revision',
 'education_2003_revision',
 'education_reporting_flag',
 'month_of_death',
 'sex',
 'detail_age_type',
 'detail_age',
 'age_substitution_flag',
 'age_recode_52',
 'age_recode_27',
 'age_recode_12',
 'infant_age_recode_22',
 'place_of_death_and_decedents_status',
 'marital_status',
 'day_of_week_of_death',
 'current_data_year',
 'injury_at_work',
 'manner_of_death',
 'method_of_disposition',
 'autopsy',
 'activity_code',
 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_',
 'icd_code_10th_revision',
 '358_cause_recode',
 '113_cause_recode',
 '130_infant_cause_recode',
 '39_cause_recode',
 'number_of_entity_axis_conditions',
 'entity_condition_1',
 'entity_condition_2',
 'entity_condition_3',
 'entity_condition_4',
 'entity_condition_5',
 'entity_condition_6',
 'entity_condition_7',
 'entity_condition_8',
 'entity_condition_9',
 'entity_condition_10',
 'entity_condition_11',
 'entity_condition_12',
 'entity_condition_13',
 'entity_co

In [7]:
# Dropping Columns
MergeData = MergeData.drop('record_condition_1',
                           'record_condition_2',
                           'record_condition_3',
                           'record_condition_4',
                           'record_condition_5',
                           'record_condition_6',
                           'record_condition_7',
                           'record_condition_8',
                           'record_condition_9',
                           'record_condition_10',
                           'record_condition_11',
                           'record_condition_12',
                           'record_condition_13',
                           'record_condition_14',
                           'record_condition_15',
                           'record_condition_16',
                           'record_condition_17',
                           'record_condition_18',
                           'record_condition_19',
                           'record_condition_20')

MergeData = MergeData.drop('113_cause_recode',
                           '130_infant_cause_recode',
                           '39_cause_recode',
                           'number_of_entity_axis_conditions',
                           'entity_condition_1',
                           'entity_condition_2',
                           'entity_condition_3',
                           'entity_condition_4',
                           'entity_condition_5',
                           'entity_condition_6',
                           'entity_condition_7',
                           'entity_condition_8',
                           'entity_condition_9',
                           'entity_condition_10',
                           'entity_condition_11',
                           'entity_condition_12',
                           'entity_condition_13',
                           'entity_condition_14',
                           'entity_condition_15',
                           'entity_condition_16',
                           'entity_condition_17',
                           'entity_condition_18',
                           'entity_condition_19')

MergeData = MergeData.drop('icd_code_10th_revision',
                           'age_recode_27',
                           'age_recode_12',
                           'detail_age',
                           'entity_condition_20',
                           'education_2003_revision',
                           'education_1989_revision')

MergeData = MergeData.filter((MergeData.method_of_disposition == 'B') | (MergeData.method_of_disposition == 'C' ))

MergeData = MergeData.drop('detail_age_type',
                           'age_substitution_flag',
                           'age_substitution_flag',
                           'infant_age_recode_22',
                           'day_of_week_of_death',
                           'current_data_year',
                           '358_cause_recode',
                           'number_of_record_axis_conditions',
                           'hispanic_origin',
                           'race_recode_5')

In [30]:
#Columns of MergeData
MergeData.columns

['resident_status',
 'education_reporting_flag',
 'month_of_death',
 'sex',
 'age_recode_52',
 'place_of_death_and_decedents_status',
 'marital_status',
 'injury_at_work',
 'manner_of_death',
 'method_of_disposition',
 'autopsy',
 'activity_code',
 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_',
 'race',
 'bridged_race_flag',
 'race_imputation_flag',
 'race_recode_3',
 'hispanic_originrace_recode']

In [8]:
categoricalColumns = ['resident_status',
                      'month_of_death',
                      'sex',
                      'age_recode_52',
                      'place_of_death_and_decedents_status',
                      'marital_status',
                      'injury_at_work',
                      'manner_of_death',
                      'autopsy',
                      'activity_code',
                      'place_of_injury_for_causes_w00_y34_except_y06_and_y07_',
                      'race',
                      'race_recode_3',
                      'hispanic_originrace_recode']

In [9]:
# Null Imputations
MergeData = MergeData.fillna({'place_of_injury_for_causes_w00_y34_except_y06_and_y07_': 12})
MergeData = MergeData.fillna({'activity_code': 11})
MergeData = MergeData.fillna({'manner_of_death': 999})
MergeData = MergeData.fillna({'place_of_death_and_decedents_status': 999})

In [10]:
# Merging all years data into dataframe
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

JoinedData=unionAll(year2005,
                    year2006,
                    year2007,
                    year2008,
                    year2009,
                    year2010,
                    year2011,
                    year2012,
                    year2013,
                    year2014,
                    year2015)

display(JoinedData)

DataFrame[resident_status: int, education_1989_revision: int, education_2003_revision: int, education_reporting_flag: int, month_of_death: int, sex: string, detail_age_type: int, detail_age: int, age_substitution_flag: int, age_recode_52: int, age_recode_27: int, age_recode_12: int, infant_age_recode_22: int, place_of_death_and_decedents_status: int, marital_status: string, day_of_week_of_death: int, current_data_year: int, injury_at_work: string, manner_of_death: int, method_of_disposition: string, autopsy: string, activity_code: int, place_of_injury_for_causes_w00_y34_except_y06_and_y07_: int, icd_code_10th_revision: string, 358_cause_recode: int, 113_cause_recode: int, 130_infant_cause_recode: int, 39_cause_recode: int, number_of_entity_axis_conditions: int, entity_condition_1: string, entity_condition_2: string, entity_condition_3: string, entity_condition_4: string, entity_condition_5: string, entity_condition_6: string, entity_condition_7: string, entity_condition_8: string, enti

In [11]:
# Replacing null values in manner_of_death
JoinData = JoinedData.fillna({'manner_of_death': 12})
JoinData = JoinedData.filter(JoinedData.manner_of_death == '2')

In [13]:
# Creating joint table on new joined data which is modified
JoinData.registerTempTable("jointTable")

In [36]:
# Query 8 - Total suicides committed 2005-2015
results_suicides = spark.sql(
  """SELECT current_data_year AS year,
            CASE manner_of_death
            WHEN '2' THEN 'Suicide'
            END AS suicides,
            COUNT(*) AS counts
     FROM jointTable
     GROUP BY 1, 2
     ORDER BY 1, 3""")

results_suicides.show()

+----+--------+------+
|year|suicides|counts|
+----+--------+------+
|2005| Suicide| 32934|
|2006| Suicide| 33562|
|2007| Suicide| 34827|
|2008| Suicide| 36251|
|2009| Suicide| 37205|
|2010| Suicide| 38710|
|2011| Suicide| 39878|
|2012| Suicide| 40929|
|2013| Suicide| 41509|
|2014| Suicide| 43139|
|2015| Suicide| 44417|
+----+--------+------+



In [38]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_suicides (ind int primary key, counts int, suicides varchar, year int)")
stmt = session.prepare("INSERT INTO results_suicides (ind, counts, suicides, year) VALUES (?, ?, ?, ?)")

for ind, item in results_suicides.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['counts'], item['suicides'], item['year']])

In [39]:
# Query 9 - Result about deaths pending
results_deaths_pending = spark.sql(
    """SELECT icd.description3 AS description,
              count(*) as counts
       FROM mergedTable mt JOIN icd10 icd ON icd.code3 = mt.icd_code_10th_revision
       WHERE mt.manner_of_death == '4'
       GROUP BY description
       ORDER BY counts DESC, description"""
)
results_deaths_pending.show()

+--------------------+------+
|         description|counts|
+--------------------+------+
|Ill-defined and u...| 42543|
|Acute myocardial ...|   294|
|      Cardiac arrest|   214|
|Hypertensive hear...|   195|
|Complications and...|   166|
|Other chronic obs...|   143|
|      Cardiomyopathy|   117|
|Cardiac arrhythmi...|    92|
|Pneumonia, unspec...|    89|
|Essential (primar...|    75|
|Sepsis, unspecifi...|    75|
|Shock, not elsewh...|    58|
|Other disorders o...|    54|
|       Other obesity|    51|
|Chronic ischemic ...|    49|
| Alzheimer's disease|    42|
|Obesity, unspecified|    38|
|           Emphysema|    36|
|Nontraumatic intr...|    33|
|  Respiratory arrest|    32|
+--------------------+------+
only showing top 20 rows



In [40]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_deaths_pending (ind int primary key, counts int, description varchar)")
stmt = session.prepare("INSERT INTO results_deaths_pending (ind, counts, description) VALUES (?, ?, ?)")

for ind, item in results_deaths_pending.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['counts'], item['description']])

In [41]:
# Query 10 - Deaths for days of the week
results_day_week = spark.sql(
    """SELECT count(*) as counts,
              CASE day_of_week_of_death
              WHEN '1' THEN 'Sunday'
              WHEN '2' THEN 'Monday'
              WHEN '3' THEN 'Tuesday'
              WHEN '4' THEN 'Wednesday'
              WHEN '5' THEN 'Thursday'
              WHEN '6' THEN 'Friday'
              WHEN '7' THEN 'Saturday'
              WHEN '9' THEN 'Unknown'
              END AS day
      FROM mergedTable
      GROUP BY day
      ORDER BY counts"""
)

results_day_week.show()

+-------+---------+
| counts|      day|
+-------+---------+
|   1196|  Unknown|
|3931885|  Tuesday|
|3933510|   Sunday|
|3939328|Wednesday|
|3945930| Thursday|
|3960250|   Monday|
|3996471|   Friday|
|4012103| Saturday|
+-------+---------+



In [42]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_day_week (ind int primary key, counts int, day varchar)")
stmt = session.prepare("INSERT INTO results_day_week (ind, counts, day) VALUES (?, ?, ?)")

for ind, item in results_day_week.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['counts'], item['day']])

In [43]:
# Query 11 - Total deaths by skin's color
results_deaths_skin_color = spark.sql(
    """SELECT count(*) as counts,
            CASE race_recode_3
            WHEN '1' THEN 'White'
            WHEN '2' THEN 'Races other than b&w'
            WHEN '3' THEN 'Black'
            END AS skin_color
       FROM mergedTable
       GROUP BY skin_color
       ORDER BY counts"""
)

results_deaths_skin_color.show()

+--------+--------------------+
|  counts|          skin_color|
+--------+--------------------+
|  759081|Races other than b&w|
| 3258257|               Black|
|23703335|               White|
+--------+--------------------+



In [45]:
# Saving data in the db
session.execute("CREATE TABLE IF NOT EXISTS results_deaths_skin_color (ind int primary key, counts int, skin_color varchar)")
stmt = session.prepare("INSERT INTO results_deaths_skin_color (ind, counts, skin_color) VALUES (?, ?, ?)")

for ind, item in results_deaths_skin_color.toPandas().iterrows():
    results = session.execute(stmt, [ind, item['counts'], item['skin_color']])

In [12]:
# Spark MLlib

# Pipeline- String Indexing, encoding and Vector Assembling
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [13]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol='method_of_disposition', outputCol="label")
stages += [label_stringIdx]

In [14]:
assemblerInputs = [c + "classVec" for c in categoricalColumns]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [15]:
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(MergeData)
preppedDataDF = pipelineModel.transform(MergeData)

In [16]:
# Choose the column that is predicted and the columns(features) that are used for prediciting
selectedcols = ["label", "features"]
dataset = preppedDataDF.select(selectedcols)

In [17]:
# Splitting the dataset into training set and test set
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)

In [18]:
# Logistic Regression model for predicting method-of-disposition (burial vs cremation)

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [19]:
predictions = lrModel.transform(testData)

In [20]:
# Comparing the predicted value against actual value
selected = predictions.select("label", "prediction", "probability")

In [21]:
# Efficiency of Model: Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.6650950346224196

In [22]:
# Accuracy: Evaluate model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", metricName = 'accuracy')
evaluator.evaluate(predictions)


0.6298288458452562